# Systolic Array Design


In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pyrtl
from pyrtl import *

from hardware_accelerators import *
from hardware_accelerators.dtypes import BaseFloat

from IPython.display import *
import numpy as np
from dataclasses import dataclass
from typing import Callable, Type, Self, List, Type

## Configuration Class


In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Callable, Type, Annotated
from enum import Enum
from pyrtl import WireVector


class ProcessingElementType(Enum):
    STANDARD = "standard"
    SIMPLE = "simple"


class SystolicArrayConfig(BaseModel):
    # Array configuration
    array_size: Annotated[
        int, Field(gt=0, description="Size N of the NxN systolic array matrix")
    ]

    # Data types
    data_type: Annotated[
        Type[BaseFloat],
        Field(
            description="Floating point format for input data and weights (e.g. Float8, BF16)"
        ),
    ]

    accum_type: Annotated[
        Type[BaseFloat],
        Field(
            description="Floating point format for accumulation, typically wider than data_type"
        ),
    ]

    # Arithmetic operations
    multiplier: Annotated[
        Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        Field(description="Floating point multiplier implementation function"),
    ]

    adder: Annotated[
        Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        Field(description="Floating point adder implementation function"),
    ]

    # Processing element configuration
    pe_type: Annotated[
        ProcessingElementType,
        Field(
            default=ProcessingElementType.STANDARD,
            description="Type of processing element to use in the array",
        ),
    ]

    pipeline_mult: Annotated[
        bool,
        Field(
            default=False,
            description="Whether to add a pipeline register after multiplication",
        ),
    ]

    class Config:
        arbitrary_types_allowed = True
        validate_assignment = True

    @property
    def data_width(self) -> int:
        """Bit width of data/weight values"""
        return self.data_type.bitwidth()

    @property
    def accum_width(self) -> int:
        """Bit width of accumulator values"""
        return self.accum_type.bitwidth()

    @classmethod
    def standard_config(
        cls,
        array_size: int,
        data_type: Type[BaseFloat],
        accum_type: Type[BaseFloat],
        multiplier: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        adder: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        pipeline_mult: bool = False,
    ) -> "SystolicArrayConfig":
        """Create a standard configuration with the STANDARD processing element type

        Args:
            array_size: Size N of the NxN systolic array
            data_type: Floating point format for input data
            accum_type: Floating point format for accumulation
            multiplier: Multiplier implementation function
            adder: Adder implementation function
            pipeline_mult: Whether to pipeline multiplication

        Returns:
            Configured SystolicArrayConfig instance
        """
        return cls(
            array_size=array_size,
            data_type=data_type,
            accum_type=accum_type,
            multiplier=multiplier,
            adder=adder,
            pipeline_mult=pipeline_mult,
            pe_type=ProcessingElementType.STANDARD,
        )

    def model_post_init(self) -> None:
        """Additional validation after initialization"""
        if self.accum_width < self.data_width:
            raise ValueError(
                f"Accumulator width ({self.accum_width}) must be >= "
                f"data width ({self.data_width})"
            )


SystolicArrayConfig(
    array_size=4,
)

# Weight Stationary Dataflow

like TPU architecture


## Simple Processing Element


We will use a weight stationary dataflow in our systolic array. Since weights need to be updated less frequently than the inputs or activations flowing through them, this will reduce the amount of memory IO and therefore the power requirements.

To start, lets focus on building the most simple version of the processing element. Each element in the array will have the following inputs and outputs:

- activation value
- weight value
- partial sum

Activations will flow from left to right, weights will flow from top to bottom, and both are passed through unchanged.  
The accumulated value output will be the result of adding the accumulation input with the product of the current activation and weight.


In [5]:
@dataclass
class PEOutputs:
    """Container for PE outputs to make connections clear"""

    data: Register
    weight: Register
    accum: Register

In [6]:
class SimpleProcessingElement:
    def __init__(
        self,
        data_type: Type[BaseFloat],
        accum_type: Type[BaseFloat],
        multiplier: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        adder: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        *args,
        **kwargs
    ):
        """Initialize processing element hardware

        Args:
            data_type: Float type for data/weight (Float8, BF16 etc)
            accum_type: Float type for accumulation
            multiplier_type: Floating point multiplier implementation
            adder_type: Floating point adder implementation
            pipeline_mult: If True, register multiplication output before passing to accumulator
        """
        # Get bit widths from format specs
        data_width = data_type.bitwidth()
        accum_width = accum_type.bitwidth()

        # Input/output registers
        self.data_reg = Register(bitwidth=data_width)
        self.weight_reg = Register(bitwidth=data_width)
        self.accum_in = WireVector(bitwidth=accum_width)
        self.accum_reg = Register(bitwidth=accum_width)

        # Control signals
        self.weight_we = WireVector(bitwidth=1)  # Weight write enable

        # Multiply-accumulate logic
        product = multiplier(self.data_reg, self.weight_reg, data_type)

        # TODO: Add float type conversion logic to pass different bitwidths to the accumulator

        sum_result = adder(product, self.accum_in, accum_type)

        self.accum_reg.next <<= sum_result

        # Store registers in output container
        self.outputs = PEOutputs(
            data=self.data_reg, weight=self.weight_reg, accum=self.accum_reg
        )

    def connect_data(self, source: Self | WireVector):
        """Connect data input from source (PE or external input)"""
        if isinstance(source, SimpleProcessingElement):
            self.data_reg.next <<= source.outputs.data
        else:
            self.data_reg.next <<= source

    def connect_weight(self, source: Self | WireVector):
        """Connect weight input from source (PE or external input)"""
        if isinstance(source, SimpleProcessingElement):
            weight_in = source.outputs.weight
        else:
            weight_in = source

        # Conditional weight update based on enable signal
        with pyrtl.conditional_assignment:
            with self.weight_we:
                self.weight_reg.next |= weight_in

    def connect_accum(self, source: Self | WireVector):
        """Connect accumulator input from source (PE or external input)"""
        if isinstance(source, SimpleProcessingElement):
            self.accum_in <<= source.outputs.accum
        else:
            self.accum_in <<= source

    def connect_weight_enable(self, enable: WireVector):
        """Connect weight write enable signal"""
        self.weight_we <<= enable

## Processing Element with Control Signals


In [12]:
class ProcessingElement:
    def __init__(
        self,
        data_type: Type[BaseFloat],
        accum_type: Type[BaseFloat],
        multiplier: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        adder: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        pipeline_mult: bool = False,
    ):
        """Initialize processing element hardware

        Args:
            data_type: Float type for data/weight (Float8, BF16 etc)
            accum_type: Float type for accumulation
            multiplier_type: Floating point multiplier implementation
            adder_type: Floating point adder implementation
            pipeline_mult: If True, register multiplication output before passing to accumulator
        """
        self.pipeline = pipeline_mult

        # Get bit widths from format specs
        data_width = data_type.bitwidth()
        acc_width = accum_type.bitwidth()

        # Input wires
        self.data_in = WireVector(data_width)
        self.weight_in = WireVector(data_width)
        self.accum_in = WireVector(acc_width)

        # Registers
        self.data_reg = Register(data_width)
        self.weight_reg = Register(data_width)
        self.accum_reg = Register(acc_width)

        # Control signals
        self.weight_en = WireVector(1)  # Weight write enable
        self.data_en = WireVector(1)  # Enable writing to the data input register
        self.adder_en = WireVector(1)  # Enable writing to the accumulator register

        # Multiply logic
        product = multiplier(self.data_reg, self.weight_reg, data_type)
        adder_input = product

        # TODO: Add float type conversion logic to pass different bitwidths to the accumulator

        # Optionally build a pipeline register to hold the multiplier result
        if self.pipeline:
            self.mul_en = WireVector(1)
            product_reg = Register(data_width)
            adder_input = product_reg
            with conditional_assignment:
                with self.mul_en:  # Enable writing to product register
                    product_reg.next |= product

        # Add the product and previous accumulation value to get partial sum
        sum_result = adder(adder_input, self.accum_in, accum_type)

        # Enable writing to data input register
        with conditional_assignment:
            with self.data_en:
                self.data_reg.next |= self.data_in

        # Enable writing to weight input register
        with conditional_assignment:
            with self.weight_en:
                self.weight_reg.next |= self.weight_in

        # Enable writing to accumulator register
        with conditional_assignment:
            with self.adder_en:
                self.accum_reg.next |= sum_result

        # Store registers in output container
        self.outputs = PEOutputs(
            data=self.data_reg, weight=self.weight_reg, accum=self.accum_reg
        )

    def connect_data(self, source: Self | WireVector):
        """Connect data input from source (PE or external input)"""
        if isinstance(source, ProcessingElement):
            self.data_in <<= source.outputs.data
        else:
            self.data_in <<= source

    def connect_weight(self, source: Self | WireVector):
        """Connect weight input from source (PE or external input)"""
        if isinstance(source, ProcessingElement):
            self.weight_in <<= source.outputs.weight
        else:
            self.weight_in <<= source

    def connect_accum(self, source: Self | WireVector):
        """Connect accumulator input from source (PE or external input)"""
        if isinstance(source, ProcessingElement):
            self.accum_in <<= source.outputs.accum
        else:
            self.accum_in <<= source

    def connect_weight_enable(self, enable: WireVector):
        """Connect weight write enable signal"""
        self.weight_en <<= enable

    def connect_data_enable(self, enable: WireVector):
        """Connect PE enable signal. Controls writing to the data input register"""
        self.data_en <<= enable

    def connect_mul_enable(self, enable: WireVector):
        """Connect multiplier enable signal. Controls writing to the product register"""
        if self.pipeline:
            self.mul_en <<= enable
        else:
            raise Warning(
                "Pipelining is disabled. There is no product register to enable. Skipping."
            )

    def connect_adder_enable(self, enable: WireVector):
        """Connect adder enable signal. Controls writing to the accumulator register"""
        self.adder_en <<= enable

    def connect_control_signals(
        self,
        weight_en: WireVector | None = None,
        data_en: WireVector | None = None,
        mul_en: WireVector | None = None,
        adder_en: WireVector | None = None,
    ):
        """Connect control signals to the processing element

        Args:
            weight_en (WireVector): Weight write enable signal. Controls writing to the weight register
            data_en (WireVector): PE enable signal. Controls writing to the data input register
            mul_en (WireVector): Multiplier enable signal. Controls writing to the product register
            adder_en (WireVector): Adder enable signal. Controls writing to the accumulator register
        """
        if data_en is not None:
            self.connect_data_enable(data_en)
        if weight_en is not None:
            self.connect_weight_enable(weight_en)
        if mul_en is not None:
            self.connect_mul_enable(mul_en)
        if adder_en is not None:
            self.connect_adder_enable(adder_en)

        return self

In [19]:
reset_working_block()
set_debug_mode(False)


pe = ProcessingElement(BF16, BF16, lmul_fast, float_adder)


w_en, d_en, acc_en = input_list(["w_en", "d_en", "acc_en"], 1)

data_in, weight_in, acc_in = input_list(["data_in", "weight_in", "acc_in"], 16)
data_reg, weight_reg, acc_reg = output_list(["data_reg", "weight_reg", "acc_reg"], 16)

pe.connect_control_signals(weight_en=w_en, data_en=d_en, adder_en=acc_en)
pe.connect_data(data_in)

pe.connect_weight(weight_in)
pe.connect_accum(acc_in)


data_reg <<= pe.outputs.data
weight_reg <<= pe.outputs.weight
acc_reg <<= pe.outputs.accum


trace = SimulationTrace(
    [w_en, d_en, acc_en, data_in, weight_in, acc_in, data_reg, weight_reg, acc_reg]
)


sim = Simulation(tracer=trace)


inputs = {

    "w_en": [1] + [0] * 5,  # ,0,0,0,0,0],
    "d_en": [1] * 6,
    "acc_en": [0] * 6,
    "weight_in": [1, 2, 3, 4, 5, 6],
    "data_in": [1, 2, 3, 4, 5, 6],
    "acc_in": [0] * 6,
}


sim.step_multiple(inputs)
trace.print_trace()

       --- Values in base 10 ---
acc_en     0 0 0 0 0 0
acc_in     0 0 0 0 0 0
acc_reg    0 0 0 0 0 0
d_en       1 1 1 1 1 1
data_in    1 2 3 4 5 6
data_reg   0 1 2 3 4 5
w_en       1 0 0 0 0 0
weight_in  1 2 3 4 5 6
weight_reg 0 1 1 1 1 1


In [18]:
BF16(binint=128)

BF16(binary='0000000010000000', decimal=1.1754943508222875e-38, decimal_approx=1.1754943508222875e-38)

In [8]:
sim.inspect("weight_out"), sim.inspect("data_out"), sim.inspect("acc_out")

(0, 5, 0)

## Systolic Array Class


In [58]:
from typing import Literal


@dataclass
class SystolicArrayPorts:
    """Container for array I/O ports"""

    # Input ports for each row/column
    data_in: List[WireVector]  # Input activations flowing left->right
    weights_in: List[WireVector]  # Input weights flowing top->bottom
    weight_load: WireVector  # Weight load enable signal (1-bit)
    # Output ports from bottom row
    results_out: List[WireVector]  # Output partial sums


class SystolicArray:
    def __init__(
        self,
        size: int,
        data_type: Type[BaseFloat],
        accum_type: Type[BaseFloat],
        multiplier: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        adder: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        dip_dataflow: bool = True,
        pipeline_mult: bool = False,
    ):
        """Initialize systolic array hardware structure

        Args:
            size: N for NxN array
            data_type: Number format for inputs (Float8, BF16)
            accum_type: Number format for accumulation
            multiplier_type: Multiplier implementation to use
            adder_type: Adder implementation to use
            dip_dataflow: Ḏiagonal-I̱nput and P̱ermutated weight-stationary (DiP) dataflow
            pipeline_mult: Add pipeline register after multiplication
        """
        self.size = size
        self.dip = dip_dataflow
        self.pipeline_mult = pipeline_mult

        # Create interface wires
        data_width = data_type.bitwidth()
        accum_width = accum_type.bitwidth()

        # Input wires
        self.data_in = [WireVector(data_width) for _ in range(size)]
        self.weights_in = [WireVector(data_width) for _ in range(size)]
        self.results_out = [WireVector(accum_width) for _ in range(size)]

        # Control wires
        self.weight_load = WireVector(1)  # Enable writing weights, shared by all PEs
        self.data_enable_in = WireVector(1)  # Enable writing to the data input register
        self.adder_enable_in = WireVector(1)  # Enable writing to the accum register

        # Control signal registers to propogate signal down the array
        self.data_controls = [Register(1) for _ in range(size)]
        self.adder_controls = [Register(1) for _ in range(size)]
        if self.pipeline_mult:
            self.mul_enable_in = WireVector(1)  # Enable writing to the product register
            self.mult_controls = [Register(1) for _ in range(size)]

        # Create PE array
        self.pe_array = [
            [
                ProcessingElement(
                    data_type,
                    accum_type,
                    multiplier,
                    adder,
                    pipeline_mult,
                )
                for _ in range(size)
            ]
            for _ in range(size)
        ]

        # Connect PEs in systolic pattern
        self._connect_array()

        # Package ports for external access
        self.ports = SystolicArrayPorts(
            data_in=self.data_in,
            weights_in=self.weights_in,
            weight_load=self.weight_load,
            results_out=self.results_out,
        )

    def _connect_array(self):
        """Connect processing elements in systolic pattern

        Data flow patterns:
        - Activations flow left to right
        - Weights flow top to bottom
        - Partial sums flow top to bottom
        """
        for row in range(self.size):
            if row == 0:
                # Connect top row of control signals to input wire
                self.data_controls[row].next <<= self.data_enable_in
                self.adder_controls[row].next <<= self.adder_enable_in
                if self.pipeline_mult:
                    self.mult_controls[row].next <<= self.mul_enable_in
            else:
                # Connect to previous row so signals propogate with data
                self.data_controls[row].next <<= self.data_controls[row - 1]
                self.adder_controls[row].next <<= self.adder_controls[row - 1]
                if self.pipeline_mult:
                    self.mult_controls[row].next <<= self.mult_controls[row - 1]

            for col in range(self.size):
                pe = self.pe_array[row][col]

                # Connect control signals
                pe.connect_control_signals(
                    weight_en=self.weight_load,
                    data_en=self.data_controls[row],
                    adder_en=self.adder_controls[row],
                )

                if self.pipeline_mult:
                    pe.connect_mul_enable(self.mult_controls[row])

                # Connect activation input:
                # First column gets external input, others connect to PE on left
                if not self.dip:
                    if col == 0:
                        pe.connect_data(self.data_in[row])
                    else:
                        pe.connect_data(self.pe_array[row][col - 1])

                # Connect weight input:
                # First row gets external input, others connect to PE above
                if row == 0:
                    pe.connect_weight(self.weights_in[col])
                    pe.connect_accum(Const(0))
                    if self.dip:
                        pe.connect_data(self.data_in[col])
                else:
                    pe.connect_weight(self.pe_array[row - 1][col])
                    pe.connect_accum(self.pe_array[row - 1][col])
                    if self.dip:
                        pe.connect_data(self.pe_array[row - 1][col - self.size + 1])

                # Connect bottom row results to output ports
                if row == self.size - 1:
                    self.results_out[col] <<= pe.outputs.accum

    def connect_weight_load(self, source: WireVector):
        """Connect weight load enable signal"""
        self.weight_load <<= source

    def connect_data_enable(self, enable: WireVector):
        """Connect PE enable signal. Controls writing to the data input register"""
        self.data_enable_in <<= enable

    def connect_mul_enable(self, enable: WireVector):
        """Connect multiplier enable signal. Controls writing to the product register"""
        if self.pipeline_mult:
            self.mul_enable_in <<= enable
        else:
            raise Warning(
                "Pipelining is disabled. There is no product register to enable. Skipping."
            )

    def connect_adder_enable(self, enable: WireVector):
        """Connect adder enable signal. Controls writing to the accumulator register"""
        self.adder_enable_in <<= enable

    def connect_control_signals(
        self,
        weight_en: WireVector | None = None,
        data_en: WireVector | None = None,
        mul_en: WireVector | None = None,
        adder_en: WireVector | None = None,
    ):
        """Connect wires to the systolic array control signal inputs. Values propogate down a chain of registers.

        Args:
            weight_en (WireVector): Weight write enable signal. Controls shifting weights in all PEs
            data_en (WireVector): PE enable signal. Controls writing to the data input register
            mul_en (WireVector): Multiplier enable signal. Controls writing to the product register
            adder_en (WireVector): Adder enable signal. Controls writing to the accumulator register
        """
        if data_en is not None:
            self.connect_data_enable(data_en)
        if weight_en is not None:
            self.connect_weight_load(weight_en)
        if mul_en is not None:
            self.connect_mul_enable(mul_en)
        if adder_en is not None:
            self.connect_adder_enable(adder_en)

        return self

    def connect_data_input(self, row: int, source: WireVector):
        """Connect data input for specified row"""
        assert 0 <= row < self.size
        self.data_in[row] <<= source

    def connect_weight_input(self, col: int, source: WireVector):
        """Connect weight input for specified column"""
        assert 0 <= col < self.size
        self.weights_in[col] <<= source

    def connect_result_output(self, col: int, dest: WireVector):
        """Connect result output from specified column"""
        assert 0 <= col < self.size
        dest <<= self.results_out[col]

## Systolic Data Setup


In [59]:
class SystolicSetup:
    """Creates diagonal delay pattern for systolic array I/O

    For a 3x3 array, creates following pattern of registers:
    (R = register, -> = connection)

    Row 0:  [R] ------->
    Row 1:  [R]->[R] -->
    Row 2:  [R]->[R]->[R]

    - Each row i contains i+1 registers
    - Input connects to leftmost register
    - Output reads from rightmost register
    - Can be used for both input and output buffering
    """

    def __init__(self, size: int, dtype: Type[BaseFloat]):
        """Initialize delay register network

        Args:
            size: Number of rows in network
            bitwidth: Bit width of data values
        """
        self.size = size
        self.dtype = dtype
        self.bitwidth = dtype.bitwidth()

        # Create input wires for each row
        self.inputs = [WireVector(self.bitwidth) for _ in range(size)]

        # Create delay register network - more delays for lower rows
        self.delay_regs = []
        self.outputs = [WireVector(self.bitwidth) for _ in range(size)]

        for i in range(size):  # Create num rows equal to the size of systolic array
            row: List[Register] = []
            # Number of buffer registers equals row index for lower triangular config
            for j in range(i + 1):
                row.append(Register(self.bitwidth))
                if j != 0:
                    # Left most register connects to inputs, others connect to previous reg
                    row[j].next <<= row[j - 1]

            # Connect row input and output
            row[0].next <<= self.inputs[i]
            self.outputs[i] <<= row[-1]
            self.delay_regs.append(row)

    def connect_input(self, row: int, source: WireVector):
        """Connect input for specified row"""
        assert (
            len(source) == self.bitwidth
        ), f"Source bitwidth ({len(source)}) must match configured bitwidth ({self.bitwidth})"
        self.inputs[row] <<= source

    def connect_output(self, row, dest: WireVector):
        """Connect final register in a buffer row to an output destination"""
        assert (
            len(dest) == self.bitwidth
        ), f"Destination bitwidth ({len(dest)}) must match configured bitwidth ({self.bitwidth})"
        dest <<= self.outputs[row]

## Matrix Multiply Unit Top Level


In [60]:
class MatrixMultiplier:
    """Top level systolic array matrix multiplier hardware"""

    def __init__(
        self,
        size: int,
        data_type: Type[BaseFloat],
        accum_type: Type[BaseFloat],
        multiplier: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        adder: Callable[[WireVector, WireVector, Type[BaseFloat]], WireVector],
        pipeline_mult: bool = False,
    ):
        self.size = size
        self.data_type = data_type
        self.accum_type = accum_type
        self.data_width = data_type.bitwidth()
        self.accum_width = accum_type.bitwidth()

        # Create hardware components
        self.systolic_array = SystolicArray(
            size, data_type, accum_type, multiplier, adder, pipeline_mult
        )
        self.systolic_setup = SystolicSetup(size, data_type)
        self.result_buffer = SystolicSetup(size, accum_type)

        # Connect internal components
        self._connect_internal_components()

    def _connect_internal_components(self):
        """Connect systolic array to input/output buffers"""
        for i in range(self.size):
            self.systolic_array.connect_data_input(i, self.systolic_setup.outputs[i])
            self.systolic_array.connect_result_output(
                i, self.result_buffer.inputs[-i - 1]
            )

    def _validate_wire_list(
        self, wires: List[WireVector], expected_width: int, purpose: str
    ):
        """Validate a list of wires meets requirements"""
        if len(wires) != self.size:
            raise ValueError(f"{purpose} requires {self.size} wires, got {len(wires)}")
        if not all(isinstance(w, WireVector) for w in wires):
            raise TypeError(f"All {purpose} must be WireVector instances")
        if not all(w.bitwidth == expected_width for w in wires):
            raise ValueError(f"All {purpose} must have bitwidth {expected_width}")

    def _split_wide_wire(
        self, wire: WireVector, width_per_slice: int
    ) -> List[WireVector]:
        """Split a wide wire into equal slices"""
        expected_width = width_per_slice * self.size
        if wire.bitwidth != expected_width:
            raise ValueError(
                f"Wide wire must have bitwidth {expected_width}, got {wire.bitwidth}"
            )
        # Use chop instead of manual slicing
        return chop(wire, *([width_per_slice] * self.size))

    def connect_weight_enable(self, enable: WireVector):
        """Connect weight enable signal"""
        if not isinstance(enable, WireVector) or enable.bitwidth != 1:
            raise ValueError("Weight enable must be 1-bit WireVector")
        self.systolic_array.connect_weight_load(enable)

    def connect_weight(self, index: int, weight: WireVector):
        """Connect an individual weight wire to the systolic array"""
        assert len(weight) == self.data_width
        self.systolic_array.connect_weight_input(index, weight)

    def connect_data(self, index: int, data: WireVector):
        """Connect an individual data wire to the systolic setup buffer"""
        assert len(data) == self.data_width
        self.systolic_setup.connect_input(index, data)

    def connect_output(self, index: int, output: WireVector):
        """Connect an individual output wire to the result buffer"""
        assert len(output) == self.accum_width
        self.result_buffer.connect_output(-index - 1, output)

    def connect_all_weights(self, weights: WireVector | List[WireVector]):
        """Connect weight inputs either as list of wires or single wide wire"""
        if isinstance(weights, list):
            self._validate_wire_list(weights, self.data_width, "weight inputs")
            weight_wires = weights
        else:
            # Split wide wire into individual weight wires
            weight_wires = chop(weights, *([self.data_width] * self.size))

        for i, wire in enumerate(weight_wires):
            self.systolic_array.connect_weight_input(i, wire)

    def connect_all_data(self, data: WireVector | List[WireVector]):
        """Connect data inputs either as list of wires or single wide wire"""
        if isinstance(data, list):
            self._validate_wire_list(data, self.data_width, "data inputs")
            data_wires = data
        else:
            # Split wide wire into individual data wires
            data_wires = chop(data, *([self.data_width] * self.size))

        for i, wire in enumerate(data_wires):
            self.systolic_setup.connect_input(i, wire)

    def connect_all_outputs(self, results: WireVector | List[WireVector]):
        """Connect result outputs either as list of wires or single wide wire"""
        if isinstance(results, list):
            self._validate_wire_list(results, self.accum_width, "result outputs")
            result_wires = results
        else:
            # Split wide wire into individual result wires
            result_wires = chop(results, *([self.accum_width] * self.size))

        for i, wire in enumerate(result_wires):
            self.result_buffer.connect_output(-i - 1, wire)

## Simulation Class


In [61]:
class MatrixMultiplierSimulator:
    def __init__(self, matrix_multiplier: MatrixMultiplier):
        self.mmu = matrix_multiplier
        self.size = matrix_multiplier.size
        self.data_type = matrix_multiplier.data_type
        self.accum_type = matrix_multiplier.accum_type

        # Create I/O ports
        self.mmu.connect_weight_enable(Input(1, "weight_enable"))

        for i in range(self.size):
            self.mmu.connect_weight(i, Input(self.mmu.data_width, f"weight_{i}"))
            self.mmu.connect_data(i, Input(self.mmu.data_width, f"data_{i}"))
            self.mmu.connect_output(i, Output(self.mmu.accum_width, f"result_{i}"))

        # Initialize simulation
        self.sim = Simulation()
        self.sim_inputs = {
            "weight_enable": 0,
            **{f"weight_{i}": 0 for i in range(self.size)},
            **{f"data_{i}": 0 for i in range(self.size)},
        }

        self._iter_state = None
        self.result_matrix = np.zeros((self.size, self.size))

    def set_matrices(self, matrix_a: np.ndarray, matrix_b: np.ndarray):
        """Set input matrices and prepare simulation state"""
        # Verify dimensions
        assert (
            matrix_a.shape == matrix_b.shape == (self.size, self.size)
        ), f"Matrices must be {self.size}x{self.size}"

        # Convert matrices to specified datatype
        self.matrix_a = self._convert_matrix(matrix_a)
        self.matrix_b = self._convert_matrix(matrix_b)

        # Load weights into PEs
        self._load_weights()

    def calculate(self):
        while next(self):
            continue
        return self.result_matrix

    def matmul(self, matrix_a: np.ndarray, matrix_b: np.ndarray):
        self.set_matrices(matrix_a, matrix_b)
        return self.calculate()

    def _convert_matrix(self, matrix: np.ndarray) -> List[List[int]]:
        """Convert numpy matrix to list of binary values in specified datatype"""
        return [[self.data_type(x).binint for x in row] for row in matrix]

    def _load_weights(self):
        """Load weights into processing elements in reverse row order"""
        for row in reversed(range(self.size)):
            for col in range(self.size):
                self.sim_inputs[f"weight_{col}"] = self.matrix_b[row][col]
            self.sim_inputs["weight_enable"] = 1
            self.sim.step(self.sim_inputs)

        # Reset weight inputs
        for i in range(self.size):
            self.sim_inputs[f"weight_{i}"] = 0
        self.sim_inputs["weight_enable"] = 0
        self.sim.step(self.sim_inputs)

    def __iter__(self):
        """Initialize iterator state"""
        if self.matrix_a is None or self.matrix_b is None:
            raise RuntimeError("Matrices must be set before iteration")

        self._iter_state = {
            "row": self.size - 1,  # Start from last row
            "extra_cycles": self.size * 3 - 1,  # Cycles needed to flush results
            "phase": "input",  # 'input' or 'flush' phase
        }
        return self

    def __next__(self):
        """Return next simulation step results"""
        if self._iter_state is None:
            raise RuntimeError("Iterator not initialized")

        # If we're done with both phases, stop iteration
        if (
            self._iter_state["phase"] == "flush"
            and self._iter_state["extra_cycles"] == 0
        ):
            raise StopIteration

        # Handle input phase
        if self._iter_state["phase"] == "input":
            if self._iter_state["row"] < 0:
                # Transition to flush phase
                self._iter_state["phase"] = "flush"
                # Clear inputs
                for i in range(self.size):
                    self.sim_inputs[f"data_{i}"] = 0
            else:
                # Load next row of input data
                for col in range(self.size):
                    self.sim_inputs[f"data_{col}"] = self.matrix_a[
                        self._iter_state["row"]
                    ][col]
                self._iter_state["row"] -= 1

        # Handle flush phase
        if self._iter_state["phase"] == "flush":
            self._iter_state["extra_cycles"] -= 1

        # Step simulation
        self.sim.step(self.sim_inputs)

        # Return current results
        current_outputs = self.get_current_results()

        # Shift previous results down and insert new results at top
        self.result_matrix[1:] = self.result_matrix[:-1]
        self.result_matrix[0] = current_outputs

        return current_outputs

    def get_current_results(self) -> List[BaseFloat]:
        """Get current values from result output ports

        Returns:
            List of values currently present on the result output ports.
            Length will equal systolic array size (one value per column).
        """
        return [
            self.accum_type(binint=self.sim.inspect(f"result_{i}"))
            for i in range(self.size)
        ]

    def inspect_pe_array(self) -> dict[str, np.ndarray]:
        """Get current state of processing element array as dictionary of matrices

        Returns:
            Dictionary with keys 'data', 'weights', 'accum', where each value is
            a matrix showing the current values in the corresponding registers
            across the PE array
        """
        # Initialize matrices to store PE values
        data_matrix = np.zeros((self.size, self.size))
        weight_matrix = np.zeros((self.size, self.size))
        accum_matrix = np.zeros((self.size, self.size))

        # Populate matrices with current PE values
        for row in range(self.size):
            for col in range(self.size):
                pe = self.mmu.systolic_array.pe_array[row][col]

                # Convert binary values to float using appropriate data types
                data_matrix[row, col] = self.data_type(
                    binint=self.sim.inspect(pe.outputs.data.name)
                )
                weight_matrix[row, col] = self.data_type(
                    binint=self.sim.inspect(pe.outputs.weight.name)
                )
                accum_matrix[row, col] = self.accum_type(
                    binint=self.sim.inspect(pe.outputs.accum.name)
                )

        return {"data": data_matrix, "weights": weight_matrix, "accum": accum_matrix}

    def inspect_systolic_setup(self) -> str:
        """Visualize current state of systolic setup registers"""
        repr_str = ""
        for row in range(self.size):
            input_val = self.data_type(binint=self.sim.inspect(f"data_{row}"))
            repr_str += f"(input={input_val}) => "

            for reg in self.mmu.systolic_setup.delay_regs[row]:
                val = self.data_type(binint=self.sim.inspect(reg.name))
                repr_str += f"{val} -> "
            repr_str += "\n"
        return repr_str

In [64]:
reset_working_block()
set_debug_mode()

# Create hardware and simulator
mmu = MatrixMultiplier(
    size=4,
    data_type=BF16,
    accum_type=BF16,
    multiplier=lmul_simple,
    adder=float_adder,
)
simulator = MatrixMultiplierSimulator(mmu)

# Set input matrices
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
weights = np.identity(4)
simulator.set_matrices(weights, weights)

# Iterate over simulation steps
for step, results in enumerate(simulator):
    print(f"Step {step} ({simulator._iter_state}):")

    print("Systolic Setup State:")
    print(simulator.inspect_systolic_setup())

    # Get PE array state
    print("PE Array State:")
    pe_state = simulator.inspect_pe_array()
    print("Data Values:")
    print(pe_state["data"])
    print("\nWeight Values:")
    print(pe_state["weights"])
    print("\nAccumulator Values:")
    print(pe_state["accum"])

    print("Current Results:")
    print([f"{float(x):.3f}" for x in results])

    print("-" * 80 + "\n")

PyrtlError: Wires used but never driven: ['tmp54994', 'tmp54993'] 

 tmp54994/1W:
Wire Traceback, most recent call last 
  File "<frozen runpy>", line 198, in _run_module_as_main
   File "<frozen runpy>", line 88, in _run_code
   File "/home/vscode/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
   File "/home/vscode/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
   File "/home/vscode/.local/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
   File "/home/vscode/.local/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
   File "/usr/local/lib/python3.12/asyncio/base_events.py", line 640, in run_forever
    self._run_once()
   File "/usr/local/lib/python3.12/asyncio/base_events.py", line 1992, in _run_once
    handle._run()
   File "/usr/local/lib/

In [63]:
[[x.decimal_approx for x in result] for result in simulator][::-1]

[[12.75, 12.75, 12.75, 12.75],
 [13.8125, 8.5, 8.5, 8.5],
 [8.5, 13.8125, 8.5, 8.5],
 [8.5, 8.5, 13.8125, 8.5],
 [8.5, 8.5, 8.5, 13.8125],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75],
 [12.75, 12.75, 12.75, 12.75]]

In [36]:
simulator.result_matrix

array([[1.0625, 2.125 , 3.125 ],
       [4.25  , 5.25  , 6.25  ],
       [7.25  , 8.5   , 9.5   ]])

---


# DiP: (diagonal input and permutated weight stationary)


Implementing the design found in the recent paper **[DiP: A Scalable, Energy-Efficient Systolic Array for Matrix Multiplication Acceleration](https://arc.net/l/quote/wllrwuvk)**


## Weight Matrix Permutation


In [316]:
def permutate_weight_matrix(arr: np.ndarray):
    # verify matrix is square
    rows, cols = arr.shape
    permutated = np.zeros((rows, cols))
    for i in range(cols):
        for j in range(rows):
            permutated[j][i] = arr[(j + i) % rows][i]
    return permutated

In [ ]:
x = np.array([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
x, permutate_weight_matrix(x)

(array([[1, 2, 3],
        [3, 4, 5],
        [5, 6, 7]]),
 array([[1., 4., 7.],
        [3., 6., 3.],
        [5., 2., 5.]]))

## New Systolic Array


TODO: Need to combine this with other systolic array class and make the behavior parametrizeable!

This version does not require any FIFO or systolic setup making it much more efficient than the standard weight stationary approach!


In [322]:
size = 4

pe_array = [["" for _ in range(size)] for _ in range(size)]

for row in range(size):
    for col in range(size):
        pe = pe_array[row][col]

        # Connect weight input:
        # First row gets external input, others connect to PE above
        if row == 0:
            pe_array[row][col] = f"dataIn{col}"
        else:
            pe_array[row][col] = pe_array[row - 1][col - size + 1]

pe_array

[['dataIn0', 'dataIn1', 'dataIn2', 'dataIn3'],
 ['dataIn1', 'dataIn2', 'dataIn3', 'dataIn0'],
 ['dataIn2', 'dataIn3', 'dataIn0', 'dataIn1'],
 ['dataIn3', 'dataIn0', 'dataIn1', 'dataIn2']]

In [48]:
class SystolicArrayDiP(SystolicArray):
    def __init__(self, *args, **kwargs):
        """Initialize systolic array hardware structure in the DiP dataflow configuration

        Args:
            size: N for NxN array
            data_type: Number format for inputs (Float8, BF16)
            accum_type: Number format for accumulation
            multiplier_type: Multiplier implementation to use
            adder_type: Adder implementation to use
            pipeline_mult: Add pipeline register after multiplication
        """
        super().__init__(*args, **kwargs)

    def _connect_array(self):
        """Connect processing elements in the DiP pattern

        Data flow patterns:
        - Activations flow top to bottom, the last one in a row connects to the first of the next row
        - Weights flow top to bottom
        - Partial sums flow top to bottom
        """
        for row in range(self.size):
            for col in range(self.size):
                pe = self.pe_array[row][col]

                # Connect weight input:
                # First row gets external input, others connect to PE above
                if row == 0:
                    pe.connect_data(self.data_in[col])
                    pe.connect_weight(self.weights_in[col])
                else:
                    pe.connect_data(self.pe_array[row - 1][col - self.size + 1])
                    pe.connect_weight(self.pe_array[row - 1][col])

                # Connect accumulator input:
                # First row starts at 0, others connect to PE above
                if row == 0:
                    pe.connect_accum(Const(0))
                else:
                    pe.connect_accum(self.pe_array[row - 1][col])

                # Connect weight enable to all PEs
                pe.connect_weight_enable(self.weight_load)

                # Connect bottom row results to output ports
                if row == self.size - 1:
                    self.results_out[col] <<= pe.outputs.accum

In [49]:
SystolicArrayDiP(3, BF16, BF16, lmul_fast, float_adder)